In [1]:
#Importar modulos essenciais
import sqlite3, operator, codecs, csv
import arcpy
import os

In [2]:
#Declaracao de variaveis para trabalhar com variaveis temporais corretamente no arcpy
def sec2hms(seconds):
	H = int(seconds) / 3600
	t = seconds % 3600
	M = int(t) / 60
	S = t % 60
	return H,M,S

def sec2str(seconds):
    return "%02d:%02d:%02d" % sec2hms (seconds)

def hms2sec(H, M = 0, S = 0):
	return float(H) * 3600 + float(M) * 60 + float(S)

def str2sec(HMS):
	'''"H:M:S" -> seconds'''
	while HMS.count(':') < 2:
		HMS = '0:' + HMS
	return hms2sec( *HMS.split(':') )

def hmsdiff(str1, str2):
    '''Returns str1 - str2, in seconds.'''
    return str2sec(str1) - str2sec(str2)

In [3]:
#Declarar os caminhos para o stop_times a ser interpolado
stop_times_path = r'Z:\Users\guilhermeiablonovski\OneDrive_UFRGS\_quicko\gtfs_testes\MG\belohorizonte_mg\bhsuplementar-shapes\stop_times.txt'
#Declarar caminho de saida de uma base de dados SQLite que devera ser gerada no processo de interpolacao para fins de tempo de processamento
sql_output_path = r'Z:\Users\guilhermeiablonovski\OneDrive_UFRGS\_quicko\gtfs_testes\MG\belohorizonte_mg\bhsuplementar-shapes-stop-times.sql'
#Declarar caminho do novo arquivo stop_times de saida
stop_times_output_path = r'Z:\Users\guilhermeiablonovski\OneDrive_UFRGS\_quicko\gtfs_testes\MG\belohorizonte_mg\bhsuplementar-shapes-stop-times\stop_times_new.txt'


In [4]:
#Parte 1 do script: criacao da base SQLite a partir do stop_times original. Nenhuma modificacao necessaria.

class CustomError(Exception):
    pass

try:
    # Check the user's version
    ArcVersionInfo = arcpy.GetInstallInfo("desktop")
    ProductName = ArcVersionInfo['ProductName']
    
    # Collect user inputs
    stop_times_file = stop_times_path
    SQLDbase = sql_output_path
    
    
    # ----- Turn stop_times.txt into a SQL table -----
    
    # Grab the data from the stop_times.txt file and insert it into the SQL table
    arcpy.AddMessage("Inserting stop_times.txt data into SQL table...")
    col_idxs = []

    if ProductName == "ArcGISPro":
        f = open(stop_times_file, encoding="utf-8-sig")
    else:
        f = open(stop_times_file)

    # Get the csv data
    reader = csv.reader(f)
    if ProductName == "ArcGISPro":
        reader = ([x.strip() for x in r] for r in reader if len(r) > 0)
    else:
        reader = ([x.decode('utf-8-sig').strip() for x in r] for r in reader if len(r) > 0)
    columns = [name.strip() for name in next(reader)]
    
    # Make sure the necessary columns are there to start with
    relevant_cols = {"trip_id": "trip_id CHAR,\n",
                    "arrival_time": "arrival_time CHAR,\n",
                    "departure_time": "departure_time CHAR,\n",
                    "stop_id": "stop_id CHAR,\n",
                    "stop_sequence": "stop_sequence INT,\n"}
    table_schema = "sqliteprimarykeyid INTEGER PRIMARY KEY,\n"
    for col in relevant_cols:
        if not col in columns:
            arcpy.AddError("Your GTFS dataset's stop_times.txt file is missing a required column: %s" % col)
            raise CustomError

    # Create the SQL database and table with the appropriate schema
    for col in columns:
        if col in relevant_cols:
            table_schema += relevant_cols[col]
        elif col == "timepoint":
            table_schema += col + " INT,\n"
        else:
            table_schema += col + " CHAR,\n"
    table_schema = table_schema.strip(",\n")
    conn = sqlite3.connect(SQLDbase)
    conn.execute("DROP TABLE IF EXISTS stop_times;")
    create_stmt = "CREATE TABLE stop_times (%s);" % table_schema
    conn.execute(create_stmt)
    conn.commit()
    
    # Add the stop_times data to the SQL table
    values_placeholders = ["?"] * len(columns)
    c = conn.cursor()
    c.executemany("INSERT INTO stop_times (%s) VALUES (%s);" %
                        (",".join(columns),
                        ",".join(values_placeholders))
                        , reader)
    conn.commit()
    
    # Add the optional timepoint column if it isn't already there
    if not "timepoint" in columns:
        c.execute("ALTER TABLE stop_times ADD timepoint INT;")

    # Create indices to speed up searching later
    c.execute("CREATE INDEX idx_arrivaltime ON stop_times (arrival_time);")
    c.execute("CREATE INDEX idx_departuretime ON stop_times (departure_time);")
    c.execute("CREATE INDEX idx_arrivaltime_stopid ON stop_times (stop_id, arrival_time);")
    c.execute("CREATE INDEX idx_tripid ON stop_times (trip_id);")
    conn.commit()

    f.close()

    # ----- Analyze the stop_times.txt file to try to understand how it's constructed -----
    
    # Gather some information about the data
    arcpy.AddMessage("Analyzing the stop_times.txt data...")
    
    # Make sure there are at least some non-blank values
    CountStmt = "SELECT COUNT(sqliteprimarykeyid) FROM stop_times WHERE arrival_time!=''"
    c.execute(CountStmt)
    numnotblank = c.fetchone()[0]
    if numnotblank == 0:
        arcpy.AddError("Your stop_times.txt does not contain any arrival_time values \
that are not blank.  No interpolation will be possible because there is no data to \
start with!")
        raise CustomError
    
    # Count total number of unique trip_ids
    CountStmt = "SELECT COUNT (DISTINCT trip_id) FROM stop_times;"
    c.execute(CountStmt)
    numtrips = c.fetchone()[0]
    arcpy.AddMessage("Total number of trip_ids: %s" % str(numtrips))
    
    # Count unique trip_ids with blank arrival_times
    CountStmt = "SELECT COUNT(DISTINCT trip_id) FROM stop_times WHERE arrival_time='';"
    c.execute(CountStmt)
    numblanktrips = c.fetchone()[0]
    arcpy.AddMessage("Number of trip_ids with blank arrival_times: %s" % numblanktrips)

    # If there were no blank arrival_times, then perhaps we need to look at departure_times.
    use_departures = False
    if numblanktrips == 0:
        use_departures = True
        # Count unique trip_ids with blank departure_times
        CountStmt = "SELECT COUNT(DISTINCT trip_id) FROM stop_times WHERE departure_time='';"
        c.execute(CountStmt)
        numblanktrips = c.fetchone()[0]
        arcpy.AddMessage("Number of trip_ids with blank departure_times: %s" % numblanktrips)

        # If there are also no blank departure_times, then this data doesn't need to be fixed.
        if numblanktrips == 0:
            arcpy.AddWarning("Your stop_times.txt file does not contain any blank arrival_time \
or departure_time values. No interpolation is needed.")
            raise CustomError
    
    # Calculate the percent of trips which have blank stop times and then make some assumptions
    percentblanktrips = round((float(numblanktrips) / float(numtrips)) * 100, 1)
    arcpy.AddMessage("Percent of trips with blank stop times: %s%%" % str(percentblanktrips))
    time_points_message = "Because %s of your trips contain blank stop times, it is likely that your GTFS dataset was \
intentionally constructed using time points.  The stops in between time points are not given a specific \
stop time value because the the arrival and departure time are not guaranteed to be consistent or exact. A lot of \
interpolation will be necessary for this data, and the procedure could take some time."
    if percentblanktrips == 100:
        arcpy.AddMessage(time_points_message % "all")
    elif percentblanktrips >= 80: # Note: this limit is fairly arbitrary
        arcpy.AddMessage(time_points_message % "most")

    # If only some trips contain blank stop times, then it's less obvious why the data contains blanks.
    # It could be a mistake, it could be a merged dataset, or it could be something else.
    else:
        # Count total number of unique stop_ids
        CountStmt = "SELECT COUNT (DISTINCT stop_id) FROM stop_times;"
        c.execute(CountStmt)
        numstops = c.fetchone()[0]
        arcpy.AddMessage("Total number of stop_ids: %s" % str(numstops))
        
        # Count unique stop_ids with blank times
        stop_time = "arrival_time"
        if use_departures:
            stop_time = "departure_time"
        CountStmt = "SELECT COUNT (DISTINCT stop_id) FROM stop_times WHERE %s='';" % stop_time
        c.execute(CountStmt)
        numblankstops = c.fetchone()[0]
        arcpy.AddMessage("Number of stop_ids with blank %s: %s" % (stop_time, numblankstops))
        
        percentblankstops = round((float(numblankstops) / float(numstops)) * 100, 1)
        arcpy.AddMessage("Percent of stops with blank stop times: %s%%" % str(percentblankstops))
        
        # Take some more guesses
        if percentblankstops <= 10: # Note: this limit is fairly arbitrary
            arcpy.AddMessage("Because the number of stops with blank stop time values is small, it is likely that \
these stop time values are blank by mistake and that your GTFS dataset was not constructed this way \
intentionally. Simple interpolation should correct these mistakes fairly quickly.")
        # Sometimes, it may be impossible to take a reasonable guess
        else:
            arcpy.AddMessage("It is unclear why some trips and stops in your GTFS dataset have blank stop time values \
and others do not.  It is possible that your GTFS dataset contains data merged from multiple transit \
systems, some of which assign values to all stop times and some of which intentionally leave stop time \
values blank in order to represent time points.  Interpolation can be used to estimate blank stop time values.")

    conn.close()

except CustomError:
    pass

except:
    raise

In [5]:
#Parte 2 do script: Interpolacao dos stop_times. Nenhuma modificacao necessaria.
class CustomError(Exception):
    pass


def interpolate_times(time_point_1, time_point_2, blank_times):
    ''' Simple interpolation method. Assume stop times are evenly spaced between time points.'''
    # [arrival_time, departure_time, id]
    num_blank_stops = len(blank_times)
    # Find the total number of seconds between departing the first time point and arriving at the second time point.
    total_time_interval_secs = hmsdiff(time_point_2[0], time_point_1[1])
    # Find the interval size that divides the total time evenly.
    time_between_stops_secs = total_time_interval_secs / float(num_blank_stops + 1)
    # Increment the interval and assign values to the blank stops
    current_secs = str2sec(time_point_1[1])
    for blank_time in blank_times:
        current_secs += time_between_stops_secs
        blank_time[0] = sec2str(current_secs)
        # Set arrival_time and departure_time to the same value.
        blank_time[1] = blank_time[0]
    return blank_times
    
    
try:
    
    # Check the user's version
    ArcVersionInfo = arcpy.GetInstallInfo("desktop")
    ProductName = ArcVersionInfo['ProductName']

    # Gather inputs
    SQLDbase = sql_output_path
    outStopTimesFile = stop_times_output_path


    # ----- First add values for anything that has only arrival_time or only departure_time blank -----

    arcpy.AddMessage("Analyzing data...")

    conn = sqlite3.connect(SQLDbase)
    c = conn.cursor()
    
    # Just set them equal to one another
    CountEasyOnesStmt = "SELECT COUNT(sqliteprimarykeyid) FROM stop_times WHERE arrival_time='' and departure_time!=''"
    c.execute(CountEasyOnesStmt)
    numeasyones = c.fetchone()[0]
    if numeasyones > 0:
        arcpy.AddMessage("There are %s stop time values that have a departure_time but not an arrival_time. For \
these cases, the arrival_time will simply be set equal to the departure_time value." % str(numeasyones))
        UpdateEasyOnesStmt = "UPDATE stop_times SET arrival_time=departure_time WHERE arrival_time='' and departure_time!=''"
        c.execute(UpdateEasyOnesStmt)
        conn.commit()
    
    CountEasyOnesStmt = "SELECT COUNT(sqliteprimarykeyid) FROM stop_times WHERE arrival_time!='' and departure_time=''"
    c.execute(CountEasyOnesStmt)
    numeasyones = c.fetchone()[0]
    if numeasyones > 0:
        arcpy.AddMessage("There are %s stop time values that have a arrival_time but not a departure_time. For \
these cases, the departure_time will simply be set equal to the arrival_time value." % str(numeasyones))
        UpdateEasyOnesStmt = "UPDATE stop_times SET departure_time=arrival_time WHERE arrival_time!='' and departure_time=''"
        c.execute(UpdateEasyOnesStmt)
        conn.commit()

    # Make sure there are at least some non-blank values
    CountStmt = "SELECT COUNT(sqliteprimarykeyid) FROM stop_times WHERE arrival_time!=''"
    c.execute(CountStmt)
    numnotblank = c.fetchone()[0]
    if numnotblank == 0:
        arcpy.AddError("Your stop_times.txt does not contain any arrival_time values \
that are not blank.  No interpolation will be possible because there is no data to \
start with!")
        raise CustomError

    # Count unique trip_ids with blank times
    CountStmt = "SELECT DISTINCT trip_id FROM stop_times WHERE arrival_time='';"
    c.execute(CountStmt)
    blanktrips = [trip[0] for trip in c.fetchall()]
    numblanktrips = len(blanktrips)
    if numblanktrips == 0:
        arcpy.AddMessage("There are no blank stop times in your stop_times.txt file, so there is no further work to be done!")
        raise CustomError
    arcpy.AddMessage("Number of trip_ids with blank arrival_times: %s" % numblanktrips)


    # ----- Interpolate blank times for all trips that have them -----
    
    arcpy.AddMessage("Interpolating blank stop times...")

    # Do some accounting to print a progress report
    tenperc = 0.1 * numblanktrips
    progress = 0
    perc = 10    
    
    # Do each trip one by one
    badtrips = []
    for trip in blanktrips:
        progress += 1
        if progress >= tenperc:
            arcpy.AddMessage(str(perc) + "% finished")
            perc += 10
            progress = 0
        
        # Get all stop_times associated with this trip.
        GetTripInfoStmt = "SELECT sqliteprimarykeyid, stop_sequence, arrival_time, departure_time FROM stop_times WHERE trip_id='%s';" % trip
        c.execute(GetTripInfoStmt)
        tripinfo = [list(trip) for trip in c.fetchall()]
        tripinfo.sort(key=operator.itemgetter(1))
        
        # Check that the first and last stop_times are not blank.
        if not tripinfo[0][2] or not tripinfo[-1][2]:
            badtrips.append(trip)
            continue
        
        # Figure out which are the time points and which are blank, and split them into groups to interpolate
        time_point_1 = ""
        time_point_2 = ""
        current_blank_times = []
        updated_tripinfo = [] # [arrival_time, departure_time, id]
        for stop in tripinfo:
            stop_formatted = [stop[2], stop[3], stop[0]]
            if stop_formatted[0]: # We found a time point
                if not time_point_1:
                    # This is the first time point in the trip
                    time_point_1 = stop_formatted
                elif not time_point_2:
                    # We have encountered the next time piont
                    time_point_2 = stop_formatted
                    # Calculate the interpolated stop times
                    updated_tripinfo += interpolate_times(time_point_1, time_point_2, current_blank_times)
                    # Prepare for the next segment
                    current_blank_times = []
                    time_point_1 = stop_formatted # time_point_2 becomes time_point_1 of the next segment
                    time_point_2 = ""
            else:
                # The time was blank. Append it to our list to deal with later.
                current_blank_times.append(stop_formatted)
            
        # Update SQL table with the interpolated values
        UpdateStmt = "UPDATE stop_times SET arrival_time=?,departure_time=?,timepoint=0 WHERE sqliteprimarykeyid=?"
        c.executemany(UpdateStmt, updated_tripinfo)
        conn.commit()
    
    # Throw a warning or error about any bad trips
    if badtrips:
        missing_timepoint_msg = "Blank stop time values could not be interpolated for %s of the trips \
in your dataset. %s trips with blank stop time values were missing times for the first stop, the last stop, or both."
        numbadtrips = len(badtrips)
        if numbadtrips == numblanktrips:
            arcpy.AddError(missing_timepoint_msg % ("any", "All"))
            raise CustomError
        else:
            missing_timepoint_msg = missing_timepoint_msg % ("some", str(numbadtrips) + " of " + str(numblanktrips)) + " Bad trips: "
            if numbadtrips < 10:
                missing_timepoint_msg += str(badtrips)
            else:
                missing_timepoint_msg += "(Showing the first 10) "
                missing_timepoint_msg += str(badtrips[0:10])
            arcpy.AddWarning(missing_timepoint_msg)


    # ----- Write stop_times back out to a csv file -----
    
    arcpy.AddMessage("Writing new stop_times.txt file...")

    def WriteStopTimesFile(f):
        wr = csv.writer(f)

        # Get the columns for stop_times.txt.
        c.execute("PRAGMA table_info(stop_times)")
        stoptimes_table_info = c.fetchall()
        columns = ()
        for col in stoptimes_table_info:
            if col[1] != "sqliteprimarykeyid":
                columns += (col[1],)
        # Write the columns to the CSV
        wr.writerow(columns)

        # Read in the rows from the stop_times SQL table
        columnquery = ", ".join(columns)
        selectstoptimesstmt = "SELECT %s FROM stop_times;" % columnquery
        c.execute(selectstoptimesstmt)
        for stoptime in c:
            # Encode in utf-8.
            if ProductName == "ArcGISPro":
                stoptimelist = [t for t in stoptime]
            else:
                stoptimelist = [t.encode("utf-8") if isinstance(t, basestring) else t for t in stoptime]
            stoptimetuple = tuple(stoptimelist)
            wr.writerow(stoptimetuple)

    # Open the new stop_times CSV for writing
    if ProductName == "ArcGISPro":
        with codecs.open(outStopTimesFile, "wb", encoding="utf-8") as f:
            WriteStopTimesFile(f)
    else:         
        with open(outStopTimesFile, "wb") as f:
            WriteStopTimesFile(f)

    arcpy.AddMessage("Done! Your updated stop_times.txt file is located at %s" % outStopTimesFile)

    conn.close()

except CustomError:
    pass

except:
    raise